# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

6087ab70c7d14c2cb1c428082283abae


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,revda,56.8010,59.9303,1.65,97,100,3.89,RU,1699505208
1,1,richards bay,-28.7830,32.0377,19.48,79,100,0.77,ZA,1699505162
2,2,yeniseysk,58.4497,92.1703,-8.39,70,7,3.96,RU,1699505208
3,3,puerto aysen,-45.4000,-72.7000,6.10,98,92,1.65,CL,1699505208
4,4,gadzhiyevo,69.2551,33.3362,-8.76,98,86,2.47,RU,1699505208


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_vacay_df = city_data_df[(city_data_df["Cloudiness"] <= 30) & 
                              (city_data_df["Humidity"] <= 85) & 
                              (city_data_df["Max Temp"] >= 20)]

# Drop any rows with null values
city_data_df.dropna
# Display sample data
ideal_vacay_df.head()

68

### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacay_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,blackmans bay,-43.0167,147.3167,21.96,66,30,2.63,AU,1699505209,
27,27,broken hill,-31.9500,141.4333,34.21,15,2,5.35,AU,1699505212,
32,32,arteche,12.2645,125.4048,29.75,69,15,3.14,PH,1699505213,
44,44,port lincoln,-34.7333,135.8667,21.36,55,9,2.90,AU,1699505215,
57,57,faya-largeau,17.9257,19.1043,22.96,29,25,6.22,TD,1699505217,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    #hotel_df.loc[index, "Lat"] = response["results"][0]["lat"]
    #hotel_df.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
broken hill - nearest hotel: Ibis Styles
arteche - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
faya-largeau - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
qandala - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
sarupathar - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
saint-joseph - nearest hotel: PALM Hotel & Spa
port douglas - nearest hotel: Mantra Aqueous Port Douglas
al fuwayliq - nearest hotel: فندق السلام
cabedelo - nearest hotel: No hotel found
khetri - nearest hotel: Saini Hotel
urangan - nearest hotel: Ramada
carnarvon - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
mahajanga - nearest hotel: Vahiny Hotel
bamboo flat - nearest hotel: Hotel Megapode Nest
san andres - nearest hotel: Hotel Decameron Isleño
gilmer - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the D

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,blackmans bay,-43.0167,147.3167,21.96,66,30,2.63,AU,1699505209,Villa Howden
27,27,broken hill,-31.9500,141.4333,34.21,15,2,5.35,AU,1699505212,Ibis Styles
32,32,arteche,12.2645,125.4048,29.75,69,15,3.14,PH,1699505213,No hotel found
44,44,port lincoln,-34.7333,135.8667,21.36,55,9,2.90,AU,1699505215,Boston Hotel
57,57,faya-largeau,17.9257,19.1043,22.96,29,25,6.22,TD,1699505217,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
525,525,warrenton,-28.1140,24.8475,21.74,14,0,3.66,ZA,1699505322,No hotel found
536,536,howrah,22.5892,88.3103,28.00,61,1,0.00,IN,1699505324,Sun India Guest House
540,540,salalah,17.0151,54.0924,28.05,36,0,5.14,OM,1699505324,Muscat International Hotel
559,559,aswan,24.0934,32.9070,24.61,69,0,5.14,EG,1699505328,Yaseen hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)